In [ ]:
# Note that in this demo, we are using PyRender NOT Blender due to constraints since this is run headlessly
# If you found this useful, feel free to cite our work!

## Downloading and Setting up SynthDa repo (Optional if you have your own models you'd like to use instead )

In [ ]:
# Create a local directory for autosynthda project
!mkdir -p ~/autosynthda/indiv

# Clone the synthda repo into that directory
!git clone https://github.com/NVIDIA/synthda ~/autosynthda/indiv


## Install Requirements for each new runtime

In [ ]:
# Change directory (Jupyter magic `%cd` is okay)
%cd ~/autosynthda/indiv/components

# List files (needs `!`)
!ls


!find ~/autosynthda/indiv/components -name requirements.txt


In [ ]:
!pip install -r ~/autosynthda/indiv/components/requirements.txt

#in Brev sometimes there is only python3 so sync the 2 with a link
!sudo ln -s /usr/bin/python3 /usr/bin/python


In [ ]:
!pip install \
    numpy==1.23.5 \
    yacs \
    filterpy \
    smplx==0.1.28 \
    trimesh==3.9.0 \
    chumpy==0.70 \
    python-dotenv


In [ ]:
# Uninstall broken/conflicting versions
!pip uninstall -y numpy matplotlib

# Reinstall compatible versions
!pip install numpy==1.24.4 matplotlib==3.7.2

!pip install spacy


In [ ]:
# hardcoded fix for chumpy for python 3.11

import inspect

# Monkey patch getargspec for chumpy compatibility with Python ≥3.11
if not hasattr(inspect, 'getargspec'):
    from collections import namedtuple

    ArgSpec = namedtuple('ArgSpec', 'args varargs keywords defaults')

    def getargspec(func):
        sig = inspect.signature(func)
        args = []
        varargs = None
        keywords = None
        defaults = []

        for name, param in sig.parameters.items():
            if param.kind in (param.POSITIONAL_ONLY, param.POSITIONAL_OR_KEYWORD):
                args.append(name)
                if param.default is not param.empty:
                    defaults.append(param.default)
            elif param.kind == param.VAR_POSITIONAL:
                varargs = name
            elif param.kind == param.VAR_KEYWORD:
                keywords = name

        return ArgSpec(args, varargs, keywords, tuple(defaults) if defaults else None)

    inspect.getargspec = getargspec


## Download and Install Repos [If done previously, then can skip this]

In [ ]:
# Clone required repositories into Brev local path
!git clone https://github.com/Vegetebird/StridedTransformer-Pose3D.git ~/autosynthda/indiv/StridedTransformer-Pose3D
!git clone https://github.com/EricGuo5513/text-to-motion.git ~/autosynthda/indiv/text-to-motion
!git clone https://github.com/wangsen1312/joints2smpl.git ~/autosynthda/indiv/joints2smpl

# (Optional) Clone SlowFast if needed
# !git clone https://github.com/facebookresearch/SlowFast.git ~/autosynthda/indiv/SlowFast

# Download Blender 3.0.0 into ~/autosynthda/indiv
#!wget -P ~/autosynthda/indiv https://download.blender.org/release/Blender3.0/blender-3.0.0-linux-x64.tar.xz

# Extract Blender in the same directory
#!tar -xf ~/autosynthda/indiv/blender-3.0.0-linux-x64.tar.xz -C ~/autosynthda/indiv


In [ ]:
# setup of gdown, and setting up the needed smpl neutral file if you dont already have it
import os
import subprocess
from pathlib import Path

# Ensure gdown is installed and ~/.local/bin is on PATH
subprocess.run(["pip", "install", "--user", "--quiet", "gdown"], check=True)
os.environ["PATH"] += os.pathsep + os.path.expanduser("~/.local/bin")

# Create the directory if it doesn't exist
target_path = Path("~/autosynthda/indiv/joints2smpl/smpl_models/smpl").expanduser()
target_path.mkdir(parents=True, exist_ok=True)

# Download the SMPL_NEUTRAL.pkl file
download_cmd = f"gdown --id 1KTFG5IKNKXi6Ri0UZomKBnh1Ze8uuElH -O {target_path / 'SMPL_NEUTRAL.pkl'}"
subprocess.run(download_cmd, shell=True, check=True)

# Verify download
if (target_path / "SMPL_NEUTRAL.pkl").exists():
    print("✅ SMPL_NEUTRAL.pkl successfully downloaded.")
else:
    print("❌ Download failed.")


In [ ]:
import os
os.environ["PATH"] += os.pathsep + os.path.expanduser("~/.local/bin")


# Change directory into the StridedTransformer repo
os.chdir(os.path.expanduser('~/autosynthda/indiv/StridedTransformer-Pose3D'))

# Install gdown if needed
!pip install -q gdown

# Make sure the pretrained checkpoint directory exists
os.makedirs('checkpoint/pretrained', exist_ok=True)

# Download the two refine/non-refine checkpoints if missing
if not os.path.exists('checkpoint/pretrained/refine_4365.pth'):
    !gdown https://drive.google.com/uc?id=1aDLu0SB9JnPYZOOzQsJMV9zEIHg2Uro7 \
           -O checkpoint/pretrained/refine_4365.pth

if not os.path.exists('checkpoint/pretrained/no_refine_4365.pth'):
    !gdown https://drive.google.com/uc?id=1l63AI9BsNovpfTAbfAkySo9X2MOWgYZH \
           -O checkpoint/pretrained/no_refine_4365.pth

# Ensure the demo/lib/checkpoint directory exists
os.makedirs('demo/lib/checkpoint', exist_ok=True)

# Download YOLOv3 weights
if not os.path.exists('demo/lib/checkpoint/yolov3.weights'):
    !gdown https://drive.google.com/uc?id=1gWZl1VrlLZKBf0Pfkj4hKiFxe8sHP-1C \
           -O demo/lib/checkpoint/yolov3.weights

# Download HRNet pose model
if not os.path.exists('demo/lib/checkpoint/pose_hrnet_w48_384x288.pth'):
    !gdown https://drive.google.com/uc?id=1CpyZiUIUlEjiql4rILwdBT4666S72Oq4 \
           -O demo/lib/checkpoint/pose_hrnet_w48_384x288.pth


In [ ]:
import os

# Ensure gdown works by adding .local/bin to PATH
os.environ["PATH"] += os.pathsep + os.path.expanduser("~/.local/bin")

# Change to text-to-motion directory
os.chdir(os.path.expanduser('~/autosynthda/indiv/text-to-motion'))

# Make sure checkpoints1 directory exists
os.makedirs('checkpoints1', exist_ok=True)

# Install gdown silently
!pip install -q gdown

# Download and unzip first model
if not os.path.exists('checkpoints1/model.zip'):
    !gdown --id 12liZW5iyvoybXD8eOw4VanTgsMtynCuU -O checkpoints1/model.zip
    !unzip -q checkpoints1/model.zip -d checkpoints1

# Download and unzip second model
if not os.path.exists('checkpoints1/model2.zip'):
    !gdown --id 1IgrFCnxeg4olBtURUHimzS03ZI0df_6W -O checkpoints1/model2.zip
    !unzip -q checkpoints1/model2.zip -d checkpoints1


In [ ]:
# Rename and clean up folder structure in preparation for the next steps
import os
import shutil
from pathlib import Path

ttm_dir = Path('~/autosynthda/indiv/text-to-motion').expanduser()
checkpoints_path = ttm_dir / 'checkpoints'
checkpoints1_path = ttm_dir / 'checkpoints1'

# Step 1: Force remove whatever is at 'checkpoints'
try:
    if checkpoints_path.is_symlink() or checkpoints_path.is_file():
        checkpoints_path.unlink()
        print("🗑️ Removed file/symlink 'checkpoints'")
    elif checkpoints_path.is_dir():
        shutil.rmtree(checkpoints_path)
        print("🗑️ Removed directory 'checkpoints'")
    elif checkpoints_path.exists():
        os.remove(checkpoints_path)
        print("🗑️ Removed special file 'checkpoints'")
except Exception as e:
    print(f"⚠️ Could not clean 'checkpoints': {e}")

# Step 2: Rename 'checkpoints1' → 'checkpoints'
try:
    if checkpoints1_path.exists() and checkpoints1_path.is_dir():
        checkpoints1_path.rename(checkpoints_path)
        print("✅ Renamed 'checkpoints1' → 'checkpoints'")
    else:
        print("❌ 'checkpoints1' does not exist or is not a directory.")
except Exception as e:
    print(f"❌ Rename failed: {e}")


## Run Each Component for Sanity Check

In [ ]:
# Step 0–5: Full setup with rendering and SMPL-compatible stack
!pip uninstall -y numpy

# Install NumPy 1.24.4 (compatible with matplotlib, numba, trimesh, chumpy)
!pip install numpy==1.24.4

# Install PyTorch with CUDA 12.4 (adjust index if needed)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

# Install all required packages
!pip install \
    iopath \
    fvcore \
    pytorchvideo \
    tensorboard \
    setuptools \
    torchinfo \
    opencv-python-headless \
    seaborn \
    Pillow \
    scikit-learn \
    scikit-image \
    matplotlib==3.7.2 \
    scipy==1.10.1 \
    numba \
    trimesh==3.23.5 \
    spacy \
    pyrender \
    imageio[ffmpeg] \
    pyglet \
    av \
    python-dotenv

# Download spaCy English model
!python3 -m spacy download en_core_web_sm

# Install ffmpeg system-wide (needed for .mp4 export from matplotlib/animation)
!sudo apt update && sudo apt install -y ffmpeg


In [ ]:
# AutoPatch for numpy version for text to motion
# Automatically patch old uses of `np.float` -> `float` in your codebase
import os

target_dir = "/home/ubuntu/autosynthda/indiv/text-to-motion"

for root, _, files in os.walk(target_dir):
    for fname in files:
        if fname.endswith(".py"):
            fpath = os.path.join(root, fname)
            with open(fpath, 'r') as file:
                content = file.read()
            patched = content.replace('np.float', 'float')  # or 'np.float64'
            if content != patched:
                with open(fpath, 'w') as file:
                    file.write(patched)
                print(f"✔️ Patched {fpath}")


In [ ]:
# Confirm installation
import cv2
print(f"✅ OpenCV version: {cv2.__version__}")


In [ ]:
# Check for the version of torch and whether cuda is used
import torch
print("Torch file:", torch.__file__)
print("CUDA attr exists:", hasattr(torch, "cuda"))

print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Device count:", torch.cuda.device_count())

In [ ]:
import os

# Define environment variable block
env_vars = """\
STRIDED_TRANSFORMER_PATH=~/autosynthda/indiv/StridedTransformer-Pose3D
TEXT_TO_MOTION_PATH=~/autosynthda/indiv/text-to-motion
JOINTS2SMPL_PATH=~/autosynthda/indiv/joints2smpl
SLOWFAST_PATH=~/autosynthda/indiv/SlowFast
BLENDER_BIN=~/autosynthda/indiv/blender-3.0.0-linux-x64/blender
BLENDER_ROOT=~/autosynthda/indiv/blender-3.0.0-linux-x64
BLENDER_PATH=~/autosynthda/indiv/blender-3.0.0-linux-x64/blender
"""

# Ensure parent directory exists
env_path = os.path.expanduser('~/autosynthda/indiv/synthda/components/.env')
os.makedirs(os.path.dirname(env_path), exist_ok=True)

# Write to .env file
with open(env_path, "w") as f:
    f.write(env_vars.strip())

print(f"✅ .env written to {env_path}")


In [ ]:
import os
from dotenv import load_dotenv

# Load .env file with all path variables
load_dotenv(os.path.expanduser('~/autosynthda/indiv/synthda/components/.env'))

# Make sure .local/bin is in PATH in case wget/gdown were installed via pip
os.environ["PATH"] += os.pathsep + os.path.expanduser("~/.local/bin")

# Overwrite joints2smpl/fit_seq.py with modified version from GitHub
!wget -O ~/autosynthda/indiv/joints2smpl/fit_seq.py \
  https://raw.githubusercontent.com/NVIDIA/synthda/main/colab/synthda_mods/fit_seq.py

# Overwrite text-to-motion/utils/plot_script.py with modified version from GitHub
!wget -O ~/autosynthda/indiv/text-to-motion/utils/plot_script.py \
  https://raw.githubusercontent.com/NVIDIA/synthda/main/colab/synthda_mods/plot_script.py


In [ ]:
# Test that your strided transformer is working
import os
from pathlib import Path

# Navigate to the StridedTransformer repo
strided_path = Path('~/autosynthda/indiv/StridedTransformer-Pose3D').expanduser()
os.chdir(strided_path)

# Run the visualization script
!python3 demo/vis.py --video sample_video.mp4



In [ ]:
# Test out text to motion
import os

# Switch to the text-to-motion folder
os.chdir(os.path.expanduser('~/autosynthda/indiv/text-to-motion'))

# Run the motion generation script
!python gen_motion_script.py \
    --name Comp_v6_KLD01 \
    --text_file input.txt \
    --repeat_time 1


In [ ]:
# Auto-patch the file to fix deprecated np.bool
file_path = os.path.expanduser("~/.local/lib/python3.10/site-packages/trimesh/voxel/runlength.py")
with open(file_path, "r") as f:
    lines = f.readlines()

with open(file_path, "w") as f:
    for line in lines:
        f.write(line.replace("dtype=np.bool", "dtype=bool"))

# install missing dependency where needed
!pip install h5py


In [ ]:
# Patch chumpy to fix removed np.bool/np.int/np.object/etc.
import pathlib

chumpy_init = pathlib.Path.home() / ".local/lib/python3.10/site-packages/chumpy/__init__.py"

if chumpy_init.exists():
    with open(chumpy_init, "r") as file:
        content = file.read()
    patched = content.replace(
        "from numpy import bool, int, float, complex, object, unicode, str, nan, inf",
        "from numpy import nan, inf; bool=bool; int=int; float=float; complex=complex; object=object; str=str"
    )
    with open(chumpy_init, "w") as file:
        file.write(patched)
    print("✅ Patched chumpy for NumPy >=1.24 compatibility.")
else:
    print("❌ chumpy/__init__.py not found. Check your install path.")


In [ ]:
# takes very very long fyi
import os
import importlib
import smplx

# Reload smplx in case of updates
importlib.reload(smplx)

# Switch to joints2smpl folder
os.chdir(os.path.expanduser('~/autosynthda/indiv/joints2smpl'))

# List the contents of the SMPL model directory
!ls -lh ~/autosynthda/indiv/joints2smpl/smpl_models/smpl/

# Run the SMPL fitting script on test_motion2.npy
!python3 fit_seq.py --files test_motion2.npy


In [ ]:
import shutil
import os

# Source folder: joints2smpl/demo/demo_results/test_motion2
src = os.path.expanduser('~/autosynthda/indiv/joints2smpl/demo/demo_results/test_motion2')

# Destination folder: synthda/components/renders/test_motion2
dst = os.path.expanduser('~/autosynthda/indiv/synthda/components/renders/test_motion2')

# Ensure the parent directory exists
os.makedirs(os.path.dirname(dst), exist_ok=True)

# Remove destination if it already exists
if os.path.exists(dst):
    shutil.rmtree(dst)

# Copy the entire folder
shutil.copytree(src, dst)
print("✅ Folder copied to expected Blender input location.")

# Verify
print("Exists:", os.path.exists(dst))
print("Contents:", os.listdir(dst) if os.path.exists(dst) else "Path does not exist.")


### In this Brev Instance / Colab Instance, we do not use Blender as it may be too intensive. For Blender we would recc the local version of SynthDa. For strictly demo purposes we are using PyRender, but the quality will not be the same as Blender

In [ ]:
#!/content/blender-3.0.0-linux-x64/blender -b -P /content/synthda/components/animation_pose.py -- --name <folder_with_ply_files>

# may not work on on Colab easily as Blender is intensive + running blender headlessly on Colab has some challenges.
# However you will be able to download the .fbx as well to view the animation locally.
# For Colab, the suggested alternative is to use pyrender instead, which we use in the demo function created for BREV below!



## Generating Your Own Demo Data [Trying out Real-Mix from SynthDa]

In [ ]:
#impt!! restart the kernel before AND after this cell is run

# Step 0: Completely remove local site-packages for numpy, scipy, trimesh, pyrender
!rm -rf ~/.local/lib/python3.10/site-packages/numpy*
!rm -rf ~/.local/lib/python3.10/site-packages/scipy*
!rm -rf ~/.local/lib/python3.10/site-packages/trimesh*
!rm -rf ~/.local/lib/python3.10/site-packages/pyrender*
!rm -rf ~/.cache/pip  # Optional: Clears pip cache

# Step 1: Install stable numpy first
!pip install numpy==1.24.4

# Step 2: Reinstall compiled deps against this version
!pip install \
    scipy==1.10.1 \
    matplotlib==3.7.2 \
    trimesh==3.23.5 \
    pyrender==0.1.45 \
    imageio[ffmpeg] \
    pyglet \
    av


In [ ]:
import pyrender
import os

os.environ["PYOPENGL_PLATFORM"] = "egl"
# This checks if EGL platform is available by verifying if the environment is headless
if os.environ.get("PYOPENGL_PLATFORM") == "egl":
    print("EGL is enabled via PYOPENGL_PLATFORM")
else:
    print("EGL is not enabled or PYOPENGL_PLATFORM not set")


In [ ]:
import numpy as np
from scipy.interpolate import interp1d
from pathlib import Path
import os
import sys

# Add synthda components path to sys.path (Brev local path)
components_path = os.path.expanduser("~/autosynthda/indiv/components")
sys.path.append(components_path)

from optimisation.optimisation_utils import map_h36m_to_smpl, upsample_pose_data, compute_P_opt

# used when both sources are real tracked motion data
def main_real_real(real_path_npz_1, real_path_npz_2, folder_path):
    folder_path = Path(folder_path)
    folder_path_variations = folder_path / "all_variations"
    folder_path_variations.mkdir(parents=True, exist_ok=True)

    # Output paths
    real_data_1_path = folder_path / 'output_keypoints_3d_real1.npy'
    real_data_2_path = folder_path / 'output_keypoints_3d_real2.npy'

    print(real_data_1_path)
    print(real_data_2_path)

    # Map .npz to SMPL-22 .npy
    real_data_1 = map_h36m_to_smpl(real_path_npz_1)
    np.save(real_data_1_path, real_data_1)

    real_data_2 = map_h36m_to_smpl(real_path_npz_2)
    np.save(real_data_2_path, real_data_2)

    # Load data to compare frame counts
    P_r = np.load(real_data_1_path)
    P_s = np.load(real_data_2_path)

    # Determine which array is shorter
    if P_r.shape[0] > P_s.shape[0]:
        smaller_array = real_data_2_path
        bigger_array = real_data_1_path
        max_frames = P_r.shape[0]
    else:
        smaller_array = real_data_1_path
        bigger_array = real_data_2_path
        max_frames = P_s.shape[0]

    # Upsample smaller array
    pose_data_upsampled = upsample_pose_data(str(smaller_array), target_frames=max_frames)
    extended_new_path = smaller_array.with_stem(smaller_array.stem + "_extended")
    np.save(extended_new_path, pose_data_upsampled)
    print("Upsampled shape:", pose_data_upsampled.shape)

    # Generate interpolated optimised variants
    weight_pairs = [(0.1, 0.9), (0.2, 0.8), (0.3, 0.7), (0.4, 0.6),
                    (0.5, 0.5), (0.6, 0.4), (0.7, 0.3), (0.8, 0.2), (0.9, 0.1)]

    # Decide which file is the "fixed" longer one
    if bigger_array == real_data_1_path:
        fixed = real_data_1_path
        variable = extended_new_path
    else:
        fixed = real_data_2_path
        variable = extended_new_path

    # Compute and save interpolations
    for w_A, w_B in weight_pairs:
        P_opt = compute_P_opt(str(fixed), str(variable), alpha=0.5, w_A=w_A, w_B=w_B)
        np.save(folder_path_variations / f"_euclidean_distances_wA{w_A}_wB{w_B}.npy", P_opt)


In [ ]:
import os
from pathlib import Path
import math
import trimesh
import pyrender
from PIL import Image

# Try EGL platform for headless rendering on GPU-enabled Brev VM
os.environ["PYOPENGL_PLATFORM"] = "egl"

# If EGL fails, try:
# os.environ["PYOPENGL_PLATFORM"] = "osmesa"

# ---- Set your local paths on Brev (modify where needed) ----
ply_dir = Path("~/autosynthda/indiv/components/renders/test_video_3_test_video_1_euclidean_distances_wA0.5_wB0.5").expanduser()
out_dir = Path("~/autosynthda/indiv/components/renders/testpng_video3_gen_frames").expanduser()
# --------------------------------------

# Create output directory if it doesn't exist
out_dir.mkdir(exist_ok=True, parents=True)

# Build a single scene with camera and light
scene = pyrender.Scene()
camera = pyrender.PerspectiveCamera(yfov=math.radians(50.0))
light = pyrender.DirectionalLight(color=[1.0, 1.0, 1.0], intensity=2.0)
scene.add(camera, pose=trimesh.transformations.translation_matrix([0, 0, 2]))
scene.add(light,  pose=trimesh.transformations.translation_matrix([0, 0, 2]))

# Initialize offscreen renderer
r = pyrender.OffscreenRenderer(
    viewport_width=1920,
    viewport_height=1080,
    point_size=1.0
)

# Load all .ply mesh files
files = sorted(ply_dir.glob("*.ply"))

# Render each mesh as a frame
for i, ply in enumerate(files):
    mesh = trimesh.load_mesh(ply)
    # If models need rotation (e.g., upright correction), uncomment below:
    # mesh.apply_transform(trimesh.transformations.rotation_matrix(math.radians(90), [1, 0, 0]))

    m = pyrender.Mesh.from_trimesh(mesh, smooth=False)

    # Remove previous mesh nodes
    for node in list(scene.mesh_nodes):
        scene.remove_node(node)
    scene.add(m)

    # Render and save frame
    color, _ = r.render(scene)
    Image.fromarray(color).save(out_dir / f"frame_{i:04d}.png")

    if i % 10 == 0:
        print(f"Rendered {i+1}/{len(files)} frames")

# Cleanup renderer
r.delete()
print("✅ All frames saved in", out_dir)


In [ ]:
import os
import shutil
import subprocess
import random
import numpy as np
from pathlib import Path
from itertools import combinations
from dotenv import dotenv_values

import math
import trimesh
import pyrender
from PIL import Image

# ─── Config ────────────────────────────────────────────────────────────────────
env_path = Path("~/autosynthda/indiv/synthda/components/.env").expanduser()
env = dotenv_values(env_path)

# No more KeyError here
STRIDED     = Path(env["STRIDED_TRANSFORMER_PATH"]).expanduser()
TEXT2M      = Path(env["TEXT_TO_MOTION_PATH"]).expanduser()
JOINTS2SMPL = Path(env["JOINTS2SMPL_PATH"]).expanduser()
COMP_ROOT   = Path("~/autosynthda/indiv/synthda/components").expanduser()
ANIM_REND   = COMP_ROOT / "renders"

from optimisation.optimisation_utils import map_h36m_to_smpl, upsample_pose_data, compute_P_opt

# ─── Helper: headless PLY→PNG via pyrender ──────────────────────────────────────
def render_ply_sequence(ply_folder: Path, png_out: Path,
                        width=1920, height=1080, fps=27):
    os.environ["PYOPENGL_PLATFORM"] = "egl"
    png_out.mkdir(parents=True, exist_ok=True)

    scene = pyrender.Scene()
    cam = pyrender.PerspectiveCamera(yfov=math.radians(50.0))
    light = pyrender.DirectionalLight(color=[1.0, 1.0, 1.0], intensity=2.0)
    scene.add(cam, pose=trimesh.transformations.translation_matrix([0, 0, 2]))
    scene.add(light, pose=trimesh.transformations.translation_matrix([0, 0, 2]))

    r = pyrender.OffscreenRenderer(viewport_width=width,
                                   viewport_height=height,
                                   point_size=1.0)

    ply_files = sorted(ply_folder.glob("*.ply"))
    for idx, ply in enumerate(ply_files):
        mesh = trimesh.load_mesh(str(ply))
        # Uncomment if upright correction is needed:
        # mesh.apply_transform(trimesh.transformations.rotation_matrix(math.radians(90), [1, 0, 0]))

        m = pyrender.Mesh.from_trimesh(mesh, smooth=False)
        for node in list(scene.mesh_nodes):
            scene.remove_node(node)
        scene.add(m)

        color, _ = r.render(scene)
        Image.fromarray(color).save(png_out / f"frame_{idx:04d}.png")

    r.delete()

# ─── Main pipeline ────────────────────────────────────────────────────────────
def both_real_main(*, weight_A, input_dir, output_dir, num_pairs):
    random.seed(42)
    video_dir = Path(input_dir).expanduser()
    out_root = Path(output_dir).expanduser()
    video_gen_root = video_dir.parent / f"videos_generated_real2_{weight_A}"
    video_gen_root.mkdir(parents=True, exist_ok=True)

    vids = list(video_dir.glob("*.mp4"))
    pairs = list(combinations(vids, 2))
    selection = random.sample(pairs, min(num_pairs, len(pairs)))

    for v1_path, v2_path in selection:
        v1, v2 = v1_path.name, v2_path.name
        pair_name = v1_path.stem + "_" + v2_path.stem
        pair_folder = out_root / pair_name
        pair_folder.mkdir(parents=True, exist_ok=True)

        shutil.copy(v1_path, pair_folder / v1)
        shutil.copy(v2_path, pair_folder / v2)

        # 1. StridedTransformer tracking
        for vn in (v1, v2):
            tgt = STRIDED / "demo" / "video" / vn
            tgt.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy(pair_folder / vn, tgt)
            subprocess.run(["python", "demo/vis.py", "--video", vn], cwd=STRIDED, check=True)

        # 2. Convert NPZ → SMPL .npy
        npz1 = STRIDED / "demo" / "output" / v1_path.stem / "output_3D" / "output_keypoints_3d.npz"
        npz2 = STRIDED / "demo" / "output" / v2_path.stem / "output_3D" / "output_keypoints_3d.npz"
        out1 = pair_folder / "output_keypoints_3d_real1.npy"
        out2 = pair_folder / "output_keypoints_3d_real2.npy"

        p1 = map_h36m_to_smpl(str(npz1)); np.save(out1, p1)
        p2 = map_h36m_to_smpl(str(npz2)); np.save(out2, p2)

        if p1.shape[0] < p2.shape[0]:
            p1 = upsample_pose_data(str(out1), p2.shape[0]); np.save(out1, p1)
        else:
            p2 = upsample_pose_data(str(out2), p1.shape[0]); np.save(out2, p2)

        # 3. Generate interpolated mixes
        var_folder = pair_folder / "all_variations"
        var_folder.mkdir(parents=True, exist_ok=True)

        for wA in [round(x, 1) for x in np.linspace(0.1, 0.9, 9)]:
            wB = round(1 - wA, 1)
            Popt = compute_P_opt(str(out1), str(out2), alpha=0.5, w_A=wA, w_B=wB)
            np.save(var_folder / f"euclidean_distances_wA{wA}_wB{wB}.npy", Popt)

        # 4. Pick selected mix and run joints2smpl
        target_npy = var_folder / f"euclidean_distances_wA{weight_A}_wB{round(1 - weight_A, 1)}.npy"
        if not target_npy.exists():
            raise FileNotFoundError(f"Missing variation: {target_npy}")

        dest_npy = JOINTS2SMPL / "demo" / "demo_data" / (pair_name + "_" + target_npy.name)
        dest_npy.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy(target_npy, dest_npy)

        subprocess.run([
            "python", "fit_seq.py",
            "--files", dest_npy.name,
            "--num_smplify_iters", "1"
        ], cwd=JOINTS2SMPL, check=True)

        ply_src = JOINTS2SMPL / "demo" / "demo_results" / dest_npy.stem
        if not ply_src.exists():
            raise FileNotFoundError(f"joints2smpl output not found: {ply_src}")

        # 5. Render PLY → PNG
        png_folder = ANIM_REND / f"{pair_name}_{target_npy.stem}"
        render_ply_sequence(ply_src, png_folder)

        # 6. Assemble PNGs into MP4 using ffmpeg
        out_mp4 = video_gen_root / f"{pair_name}.mp4"
        subprocess.run([
            "ffmpeg", "-y",
            "-framerate", str(27),
            "-i", str(png_folder / "frame_%04d.png"),
            "-c:v", "libx264",
            "-pix_fmt", "yuv420p",
            str(out_mp4)
        ], check=True)

        print("✅ Generated:", out_mp4)


In [ ]:
both_real_main(
    weight_A=0.2,
    input_dir="~/autosynthda/indiv/components/dataset/specific_data",
    output_dir="~/autosynthda/indiv/components/dataset/data_manipulation",
    num_pairs=1
)
